In [39]:
import mlrun

In [40]:
project = mlrun.get_or_create_project('testing-workdir','./',user_project=True,init_git=True,subpath='./project')

> 2023-03-26 08:42:19,515 [info] loaded project testing-workdir from MLRun DB


In [41]:
project.create_remote('https://github.com/GiladShapira94/test-working-dir.git')

In [42]:
print(project.to_yaml())

kind: project
metadata:
  name: testing-workdir-shapira
  created: '2023-03-26T07:22:52.792000'
spec:
  functions:
  - name: add-1
    kind: job
    image: mlrun/mlrun
    handler: add_1.add_1
  - name: add-2
    kind: job
    image: mlrun/mlrun
    handler: add_2.add_2
  workflows:
  - path: ./project/workflow.py
    name: main
  artifacts: []
  conda: ''
  source: git://github.com/GiladShapira94/test-working-dir.git#master
  subpath: ./project
  origin_url: git://github.com/GiladShapira94/test-working-dir.git#master
  load_source_on_run: false
  desired_state: online
  owner: shapira
status:
  state: online



In [43]:
%%writefile ./project/add_1.py

def add_1(x):
    x = x+1
    
    return x

Overwriting ./project/add_1.py


In [44]:
%%writefile ./project/add_2.py

def add_2(x):
    x = x+2
    
    return x

Overwriting ./project/add_2.py


In [45]:
project.set_function(name='add-1',kind='job',image='mlrun/mlrun',handler='add_1.add_1',with_repo=True)

In [46]:
project.set_function(name='add-2',kind='job',image='mlrun/mlrun',handler='add_2.add_2',with_repo=True)

In [47]:
%%writefile ./project/workflow.py


from kfp import dsl
from mlrun.platforms import auto_mount
import os
import sys
import mlrun

funcs = {}


def kfpipeline(x):
    add_1 = mlrun.run_function('add-1',params={'x':x},outputs=['return'])
    
    add_2 = mlrun.run_function('add-2',params={'x':add_1.outputs['return']})

Overwriting ./project/workflow.py


In [48]:
project.set_workflow('main','./project/workflow.py')

In [49]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('./') if isfile(join('./', f))]

In [50]:
import os

# The directory you want to start the search from
start_path = './'
files = []
# Traverse through the directory and its subdirectories
for root, dirs, files in os.walk(start_path):
    for filename in files:
        # Get the full file path
        file_path = os.path.join(root, filename)
        # Do something with the file_path, such as print it
        print(file_path)


./example_working_with_dir.ipynb
./project.yaml
./.git/HEAD
./.git/config
./.git/description
./.git/hooks/applypatch-msg.sample
./.git/hooks/commit-msg.sample
./.git/hooks/fsmonitor-watchman.sample
./.git/hooks/post-update.sample
./.git/hooks/pre-applypatch.sample
./.git/hooks/pre-commit.sample
./.git/hooks/pre-push.sample
./.git/hooks/pre-rebase.sample
./.git/hooks/pre-receive.sample
./.git/hooks/prepare-commit-msg.sample
./.git/hooks/update.sample
./.git/info/exclude
./.ipynb_checkpoints/example_working_with_dir-checkpoint.ipynb
./project/add_1.py
./project/add_2.py
./project/project.yaml
./project/workflow.py


In [51]:
files = ['./project/workflow.py','./project/add_2.py','./project/add_1.py','./example_working_with_dir.ipynb','./project/project.yaml']

In [52]:
project.set_source('git://github.com/GiladShapira94/test-working-dir.git#master')

In [53]:
project.save()

In [ ]:
project.push(branch='master',message="First Push",add=files)